<a href="https://colab.research.google.com/github/khevana98/githubtest98/blob/master/Multi_Emotion_Analysis_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install --upgrade gensim==3.6.0
!pip3 install --upgrade tensorflow

In [ ]:
import re
import string
import pickle
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Embedding

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving text_emotion.csv to text_emotion.csv


In [ ]:
df = pd.read_csv("text_emotion.csv")

In [ ]:
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [ ]:
df.drop(['tweet_id','author'],axis=1)

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [ ]:
df = df[['content','sentiment']]
df.head()

,content,sentiment
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,enthusiasm
4,@dannycastillo We want to trade with someone w...,neutral


# **1. Data Preprocessing: -**

## **=> Text Cleaning: -**

In [ ]:
def clean_tweets(text):
  text = re.sub(r'^RT[\s]+', '', text)
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'#', '', text)
  text = tokenizer.tokenize(text)
  words = []
  for word in text:
    if word not in stop_words and word not in string.punctuation:
      words.append(word)
  return words

In [ ]:
df["content"] = df["content"].apply(lambda text: clean_tweets(text))
df.head()

,content,sentiment
0,"[know, listenin, bad, habit, earlier, started,...",empty
1,"[layin, n, bed, headache, ughhh, ..., waitin, ...",sadness
2,"[funeral, ceremony, ..., gloomy, friday, ...]",sadness
3,"[wants, hang, friends, soon]",enthusiasm
4,"[want, trade, someone, houston, tickets, one]",neutral


In [ ]:
df["content"] = df["content"].apply(lambda text: [lemmatizer.lemmatize(word) for word in text])
df.head()

,content,sentiment
0,"[know, listenin, bad, habit, earlier, started,...",empty
1,"[layin, n, bed, headache, ughhh, ..., waitin, ...",sadness
2,"[funeral, ceremony, ..., gloomy, friday, ...]",sadness
3,"[want, hang, friend, soon]",enthusiasm
4,"[want, trade, someone, houston, ticket, one]",neutral


In [ ]:
sentiment_to_int = {}
int_to_sentiment = {}
s_no = 0
for i in df["sentiment"].unique():
    sentiment_to_int[i] = s_no
    int_to_sentiment[s_no] = i
    s_no+=1
sentiment_to_int, int_to_sentiment

({'anger': 12,
  'boredom': 10,
  'empty': 0,
  'enthusiasm': 2,
  'fun': 7,
  'happiness': 9,
  'hate': 8,
  'love': 6,
  'neutral': 3,
  'relief': 11,
  'sadness': 1,
  'surprise': 5,
  'worry': 4},
 {0: 'empty',
  1: 'sadness',
  2: 'enthusiasm',
  3: 'neutral',
  4: 'worry',
  5: 'surprise',
  6: 'love',
  7: 'fun',
  8: 'hate',
  9: 'happiness',
  10: 'boredom',
  11: 'relief',
  12: 'anger'})

In [ ]:
df["sentiment"] = df["sentiment"].apply(lambda text: sentiment_to_int[text])
df.head()

,content,sentiment
0,"[know, listenin, bad, habit, earlier, started,...",0
1,"[layin, n, bed, headache, ughhh, ..., waitin, ...",1
2,"[funeral, ceremony, ..., gloomy, friday, ...]",1
3,"[want, hang, friend, soon]",2
4,"[want, trade, someone, houston, ticket, one]",3


In [ ]:
word2count = {}
for sentence in df['content']:
  for word in sentence:
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1
# word2count

In [ ]:
sortedword2count = sorted(word2count.items(), key=lambda x : x[0])
# sortedword2count

In [ ]:
threshhold = 5
filtered_word_by_count = []
for word, count in sortedword2count:
  if count >= threshhold:
    filtered_word_by_count.append((word, count))
# filtered_word_by_count = filtered_word_by_count[56:1834]
print(len(filtered_word_by_count), "Words to Count: ", filtered_word_by_count)

5388 Words to Count:  [('(8', 9), ('(:', 53), ('):', 9), ('--->', 6), ('-->', 9), ('->', 12), ('. .', 19), ('. . .', 49), ('. ...', 9), ('..', 2368), ('...', 6790), ('0', 31), ('07', 5), ('08', 6), ('09', 36), ('1', 334), ('1.5', 7), ('1/2', 20), ('10', 149), ('100', 65), ('1000', 15), ('100th', 7), ('10:30', 9), ('10am', 7), ('10pm', 8), ('10th', 11), ('11', 49), ('11:11', 6), ('12', 54), ('12am', 5), ('12th', 8), ('13', 27), ('14', 35), ('140', 17), ('15', 53), ('150', 7), ('15th', 5), ('16', 27), ('167', 5), ('16th', 6), ('17', 27), ('18', 27), ('18th', 8), ('19', 19), ('1995', 5), ('1:30', 5), ('1am', 8), ('1st', 78), ('2', 953), ('2-3', 9), ('2.0', 7), ('2.5', 8), ('20', 60), ('200', 24), ('2000', 6), ('2007', 5), ('2008', 12), ('2009', 26), ('2010', 9), ('20th', 5), ('21', 24), ('22', 19), ('22nd', 6), ('23', 10), ('24', 30), ('24/7', 5), ('25', 33), ('26', 14), ('27', 9), ('27th', 5), ('28', 6), ('29', 9), ('2:30', 5), ('2am', 7), ('2b', 6), ('2day', 56), ('2moro', 19), ('2morro

In [ ]:
words_to_index = {}
index_to_words = {}
word_number = 1
for word, count in dict(filtered_word_by_count).items():
  words_to_index[word] = word_number
  index_to_words[word_number] = word
  word_number += 1
print(len(words_to_index), "Words to Index: ", words_to_index)
print(len(index_to_words), "Index to Word: ", index_to_words)

5388 Words to Index:  {'(8': 1, '(:': 2, '):': 3, '--->': 4, '-->': 5, '->': 6, '. .': 7, '. . .': 8, '. ...': 9, '..': 10, '...': 11, '0': 12, '07': 13, '08': 14, '09': 15, '1': 16, '1.5': 17, '1/2': 18, '10': 19, '100': 20, '1000': 21, '100th': 22, '10:30': 23, '10am': 24, '10pm': 25, '10th': 26, '11': 27, '11:11': 28, '12': 29, '12am': 30, '12th': 31, '13': 32, '14': 33, '140': 34, '15': 35, '150': 36, '15th': 37, '16': 38, '167': 39, '16th': 40, '17': 41, '18': 42, '18th': 43, '19': 44, '1995': 45, '1:30': 46, '1am': 47, '1st': 48, '2': 49, '2-3': 50, '2.0': 51, '2.5': 52, '20': 53, '200': 54, '2000': 55, '2007': 56, '2008': 57, '2009': 58, '2010': 59, '20th': 60, '21': 61, '22': 62, '22nd': 63, '23': 64, '24': 65, '24/7': 66, '25': 67, '26': 68, '27': 69, '27th': 70, '28': 71, '29': 72, '2:30': 73, '2am': 74, '2b': 75, '2day': 76, '2moro': 77, '2morrow': 78, '2nd': 79, '2night': 80, '2nite': 81, '2pm': 82, '3': 83, '3-4': 84, '3.0': 85, '30': 86, '300': 87, '3000': 88, '31': 89, '

In [ ]:
with open('words_to_index.pkl', 'wb') as file:
  pickle.dump(words_to_index, file)
with open('index_to_words.pkl', 'wb') as file:
  pickle.dump(index_to_words, file)

In [ ]:
pd.DataFrame(words_to_index.items(), columns=['Words', 'Index']).to_csv('words_to_index.csv')
pd.DataFrame(index_to_words.items(), columns=['Index', 'Words']).to_csv('index_to_words.csv')

In [ ]:
content_maxlen = max(df['content'], key=len)
content_maxlen = len(content_maxlen)
content_maxlen

45

In [ ]:
w2vmodel = Word2Vec(df['content'], min_count=5, vector_size=100)
# w2vmodel.wv.save_word2vec_format('tweets_word_to_vec.txt', binary=False)

TypeError: ignored

In [ ]:
def sentences_to_indices(X, word_to_index, max_len):
   
    m = X.shape[0]
    X_indices = np.zeros((m, max_len))

    for i in range(m):
        j = 0
        for w in X[i]:
          if j < max_len:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            X_indices[i, j] = word_to_index.get(w, 0)
            j += 1
        # print("Sentence", X[i], X1_indices)
    return X_indices

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.values.reshape(-1)]
    return Y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df[['content']], sentiment_one_hot)

In [ ]:
# df = pd.read_csv('text_emotion_cleaned.csv')
content_int_vec = sentences_to_indices(x_train, words_to_index, max_len = 20)
content_int_vec.shape

In [ ]:
sentiment_one_hot = convert_to_one_hot(y_train, 13)
sentiment_one_hot

# - Temp Code:

In [ ]:
# sentiment_one_hot = df[['sentiment_anger', 'sentiment_boredom', 'sentiment_empty', 'sentiment_enthusiasm', 'sentiment_fun', 'sentiment_happiness', 'sentiment_hate', 'sentiment_love', 'sentiment_neutral', 'sentiment_relief', 'sentiment_sadness', 'sentiment_surprise', 'sentiment_worry']].to_numpy()
# print(sentiment_one_hot.shape)

In [ ]:
# df = df.drop(['sentiment'],axis=1)
# df = df.join(sentiment_one_hot)
# df.head()

In [ ]:
# df.to_csv('text_emotion_cleaned.csv')

# **2. Model creation: -**

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
  
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["able"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
embedding_layer = pretrained_embedding_layer(w2vmodel.wv, words_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

In [ ]:
def emotion_analyzer(input_shape, word_to_vec_map, word_to_index):
    """
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 100-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary
    """
    
    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    sentence_indices = Input(input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with Word Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(rate=0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(rate=0.5)(X)
    # Propagate X through a Dense layer with 13 units
    X = Dense(13)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
        
    return model

In [ ]:
model = emotion_analyzer((20,), w2vmodel.wv, words_to_index)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs = 100, batch_size = 32, shuffle=True)

In [ ]:
loss, acc = model.evaluate(sentences_to_indices(x_test, words_to_index, max_len = 20), convert_to_one_hot(y_test, 13))
print()
print("Test accuracy = ", acc)

In [ ]:
pred = model.predict(sentence_to_indices(x_test, words_to_index, max_len = 20))
for i in range(len(x_test)):
    num = np.argmax(pred[i])
    if(num == actual):
      print(f'Expected: {int_to_sentiment[y_test[i]]} prediction: {x_test[i]} {int_to_sentiment[num]}')
    # elif(num != actual):
    #   print(f'Expected: {int_to_sentiment[y_test[i]]} prediction: {x_test[i]} {int_to_sentiment[num]}')

# **- Temp Code:** 

In [ ]:
# df['content'] = X1_indices
# df

In [ ]:
# df["content"] = df["content"].apply(lambda text: word_tokenize(text))
# df.head()

In [ ]:
# df["content"] = df["content"].apply(lambda text: [lemmatizer.lemmatize(word) for word in text if word not in set(stop_words)])
# df.head()

In [ ]:
# df["content"] = df["content"].apply(lambda text:" ".join([word for word in text if word not in string.punctuation]))
# df.head()

In [ ]:
# df.to_csv('text_emotion_preprocessed.csv')

In [ ]:
# df.replace()
# replace = dict()
# c=1
# for i in df["sentiment"].unique():
#     replace[i] = c
#     c+=1

In [ ]:
# df["content"] = df["content"].apply(lambda text:[porter.stem(word) for word in text])

In [ ]:
# df.head()